In [1]:
import numpy as np
import pandas as pd
import torch
from bert_model_pre import RegressorBERT
from transformers import BertTokenizer, BertConfig
from bert_training import rmse_loss
from torch import nn
import torch

/home/keddie/anaconda3/envs/hmcl_proj/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-28 12:32:13.442659: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-28 12:32:14.306103: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
config = BertConfig.from_pretrained("regressor_bert_best/")
regression_head = nn.Sequential(
        nn.Linear(768, 512, bias=True),
        nn.ReLU(),
        nn.Dropout(0.0005234335151669267,inplace=False),
        nn.Linear(512,1,bias=True)
    )
model_name = "google-bert/bert-base-multilingual-cased"
device = torch.device("cuda:0")

model = RegressorBERT(model_name="regressor_bert_best/", config=config, load_finetuned=True, aggregation_method="mean", hidden=[-1,-2], freeze_bert="all_but_last", regression_head=regression_head).to(device)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [3]:
# device = torch.device("cuda:0")

# model_name = "google-bert/bert-base-multilingual-cased"
# tokenizer = BertTokenizer.from_pretrained(model_name, return_tensors="pt", truncation=True, max_length=512, padding="max_length")
# model = RegressorBERT(model_name="regressor_bert_best/", hidden=[-1,-2],\
#                         device=device, loss_func=rmse_loss,\
#                         aggregation_method="mean")
# model.eval()


In [4]:
# from transformers import BertForSequenceClassification

# model = BertForSequenceClassification.from_pretrained(model_name, problem_type="regression", num_labels=1).to(device)

In [5]:
df = pd.read_csv("funda_data_11_06_2024.csv")
df = df[["descrip", "price"]]

In [6]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# for idx, head in enumerate(att):
#     aux_att = pd.DataFrame(head)
#     sns.heatmap(aux_att)
#     plt.title(f"Head_{idx}")
#     plt.show()

# aux_att = pd.DataFrame(np.mean(att,axis=0))
# sns.heatmap(aux_att)
# plt.title(f"Across heads avg")
# plt.show()

In [7]:
from bertviz import model_view

# limit = 50
# tokens = tokenizer.convert_ids_to_tokens(tok["input_ids"][0])[:limit]
# # (layer, attention_head, input_length, input_length)
# # reduced_att = torch.stack([torch.stack([att[:, :15, :15] for att in att_batch]) for att_batch in outputs.attentions], dim=0)
# reduced_att = torch.stack([torch.stack([att[:,:limit,:limit] for att in layer_att]) for layer_att in outputs.attentions])

# print(reduced_att.shape)

# model_view(attention=reduced_att, tokens=tokens)

In [8]:
example = df.iloc[42]
example

descrip    Op zoek naar  een  instapklare gezinswoning?  ...
price                                                 500000
Name: 42, dtype: object

In [9]:
import shap
from transformers import AutoTokenizer


def predict(x):
    tok = torch.concat([tokenizer.encode(text, padding="max_length", truncation=True, max_length=512, return_tensors="pt").to(device) for text in x], dim=0)
    outputs = model(input_ids=tok, attention_mask=None, token_type_ids=None)["value"].cpu().detach()

    return outputs

explainer = shap.Explainer(predict, tokenizer)

vals = explainer([example["descrip"]])

Token indices sequence length is longer than the specified maximum sequence length for this model (746 > 512). Running this sequence through the model will result in indexing errors
BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
PartitionExplainer explainer: 2it [00:21, 21.65s/it]               


In [10]:
# z = (x-mean)/sd => z*sd+mean=x
df.iloc[42]

descrip    Op zoek naar  een  instapklare gezinswoning?  ...
price                                                 500000
Name: 42, dtype: object

In [11]:
prices = df["price"].to_numpy()
z_scores = (prices-np.mean(prices))/np.std(prices)
pandas_df = df.iloc[z_scores < 3]

mean = np.mean(pandas_df["price"].to_numpy())
sd = np.std(pandas_df["price"].to_numpy())

In [12]:
example

def pred(x):
    tok = tokenizer(x, max_length=512, truncation=True, padding="max_length", return_tensors="pt").to(device)
    out = model(**tok)["value"]

    true_out = out*sd*mean

    return true_out.item()

pred(example["descrip"])

35673559040.0

In [13]:
from IPython.display import display, HTML

a = shap.plots.text(vals, display=False)

display(HTML(a))

In [14]:
import matplotlib.pyplot as plt
